##Analysis Questions
##### Can you categorize the factors for a sucessful and unsuccesful project? Classification
##### Can you predict if a project will succeed or fail? Logistic Regression
##### Can you predict how much a backer will pledge to a project based on various factors? Linear Regression
##### What are the most important factors for a successful project and unsuccessful project?

In [2]:
import pandas as pd
import numpy as np
import chardet

In [3]:
# Reading error with UTF-8 unable to decode the file
#ks16a = pd.read_csv("/dbfs/FileStore/tables/ks_projects_201612-284ce.csv")

In [4]:
# Code based on https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings
# Use chardet to detect character encoding: chardet shows Windows-1252 encoding

with open("/dbfs/FileStore/tables/ks_projects_201612-284ce.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
    
print(result)


{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}

In [5]:
# Read CSV file with encolding Windows-1252
ks16a = pd.read_csv("/dbfs/FileStore/tables/ks_projects_201612-284ce.csv", encoding='Windows-1252')



/databricks/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
 interactivity=interactivity, compiler=compiler, result=result)

In [6]:
ks16a.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,nan
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,nan
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,nan
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,nan
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,nan


In [7]:
ks16a.columns

Out[51]: Index(['ID ', 'name ', 'category ', 'main_category ', 'currency ', 'deadline ',
 'goal ', 'launched ', 'pledged ', 'state ', 'backers ', 'country ',
 'usd pledged ', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
 'Unnamed: 16'],
 dtype='object')

In [8]:
# Column names have a space, rename column names to have no spaces 
ks16a.columns = ['ID', 'name', 'category', 'main_category', 'currency', 'deadline', \
       'goal', 'launched', 'pledged', 'state', 'backers', 'country', \
       'usd_pledged', 'c_13', 'c_14', 'c_15', 'c_16']
    

In [9]:
# In order to create Pyspark dataframe, cast all columns to string
ks16a["ID"] = ks16a["ID"].astype(str)
ks16a["name"] = ks16a["name"].astype(str)
ks16a["category"] = ks16a["category"].astype(str)
ks16a["main_category"] = ks16a["main_category"].astype(str)
ks16a["currency"] = ks16a["currency"].astype(str)
ks16a["deadline"] = ks16a["deadline"].astype(str)
ks16a["goal"] = ks16a["goal"].astype(str)
ks16a["launched"] = ks16a["launched"].astype(str)
ks16a["pledged"] = ks16a["pledged"].astype(str)
ks16a["state"] = ks16a["state"].astype(str)
ks16a["backers"] = ks16a["backers"].astype(str) 
ks16a["country"] = ks16a["country"].astype(str)                                              
ks16a["usd_pledged"] = ks16a["usd_pledged"].astype(str)
ks16a["c_13"] = ks16a["c_13"].astype(str)
ks16a["c_14"] = ks16a["c_14"].astype(str)
ks16a["c_15"] = ks16a["c_15"].astype(str)
ks16a["c_16"] = ks16a["c_16"].astype(str)

In [10]:
# Create a dataframe in Spark
ks16 = spark.createDataFrame(ks16a)

In [11]:
# Import functions/datatypes for timestamp, integer, and double
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Drop extra columns
ks16 = ks16.drop('c_13','c_14','c_15','c_16')


# Cast from string to integer and double
ks16 = ks16.withColumn("ID", ks16['ID'].cast(IntegerType()))
ks16 = ks16.withColumn("goal", ks16['goal'].cast(IntegerType()))
ks16 = ks16.withColumn("pledged", ks16['pledged'].cast(DoubleType()))
ks16 = ks16.withColumn("backers", ks16['backers'].cast(IntegerType()))
ks16 = ks16.withColumn("usd_pledged", ks16['usd_pledged'].cast(DoubleType()))


In [12]:
from pyspark.sql.functions import isnan, when, count, col
ks16.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ks16.columns]).show()

+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+
 ID|name|category|main_category|currency|deadline|goal|launched|pledged|state|backers|country|usd_pledged|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+
 0| 0| 0| 0| 0| 0| 632| 0| 624| 0| 623| 0| 4413|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+

In [13]:
# Drop all nulls from the data frame
ks16 = ks16.dropna()

In [14]:
# Verify if there are no nulls 
from pyspark.sql.functions import isnan, when, count, col
ks16.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ks16.columns]).show()

+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+
 ID|name|category|main_category|currency|deadline|goal|launched|pledged|state|backers|country|usd_pledged|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+

In [15]:
import time
import datetime

# Convert Timestamp to Date(YYYY-MM-DD)
ks16 = ks16.withColumn("deadline", to_date(unix_timestamp("deadline", "yyyy-MM-dd").cast("timestamp")))
ks16 = ks16.withColumn("launched", to_date(unix_timestamp("launched", "yyyy-MM-dd").cast("timestamp")))

In [16]:
ks16.toPandas()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000,2015-08-11,0.00,failed,0,GB,0.00
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000,2013-01-12,220.00,failed,3,US,220.00
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000,2012-03-17,1.00,failed,1,US,1.00
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500,2015-07-04,"1,283.00",canceled,14,US,"1,283.00"
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000,2016-02-26,"52,375.00",successful,224,US,"52,375.00"
5,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000,2014-12-01,"1,205.00",successful,16,US,"1,205.00"
6,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000,2016-02-01,453.00,failed,40,US,453.00
7,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000,2014-04-24,"8,233.00",canceled,58,US,"8,233.00"
8,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000,2014-07-11,"6,240.57",canceled,43,US,"6,240.57"
9,100004721,Of Jesus and Madmen,Nonfiction,Publishing,CAD,2013-10-09,2500,2013-09-09,0.00,failed,0,CA,0.00


In [17]:
# No issues with decoding errors
ks18a = pd.read_csv("/dbfs/FileStore/tables/ks_projects_201801-a566d.csv")
ks18a.columns = ['ID', 'name', 'category', 'main_category', 'currency', 'deadline', \
       'goal', 'launched', 'pledged', 'state', 'backers', 'country', \
       'usd_pledged','usd_pledged_real', 'usd_goal_real']
ks18 = spark.createDataFrame(ks18a)

In [18]:
#Reference code: Chapter 5 Big Data Analysis
from pyspark.sql.functions import isnan, when, count, col

ks18.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ks18.columns]).show()



+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+----------------+-------------+
 ID|name|category|main_category|currency|deadline|goal|launched|pledged|state|backers|country|usd_pledged|usd_pledged_real|usd_goal_real|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+----------------+-------------+
 0| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3797| 0| 0|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+----------------+-------------+

In [19]:
# Drop all null rows
ks18 = ks18.dropna()

# Drop extra columns
ks18 = ks18.drop('usd_pledged_real','usd_goal_real')


In [20]:
# Verify there are no nulls
ks18.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ks18.columns]).show()

+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+
 ID|name|category|main_category|currency|deadline|goal|launched|pledged|state|backers|country|usd_pledged|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+

In [21]:
# Convert Timestamp to Date(yyyy-MM-dd)
ks18 = ks18.withColumn('deadline',to_date(unix_timestamp('deadline','yyyy-MM-dd').cast("timestamp"))) 
ks18 = ks18.withColumn('launched',to_date(unix_timestamp('launched','yyyy-MM-dd').cast("timestamp")))

In [22]:
display(ks18).take(5)

ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11,0.0,failed,0,GB,0.0
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02,2421.0,failed,15,US,100.0
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12,220.0,failed,3,US,220.0
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17,1.0,failed,1,US,1.0
1000011046,Community Film Project: The Art of Neighborhood Filmmaking,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04,1283.0,canceled,14,US,1283.0
1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26,52375.0,successful,224,US,52375.0
1000023410,Support Solar Roasted Coffee & Green Energy! SolarCoffee.co,Food,Food,USD,2014-12-21,1000.0,2014-12-01,1205.0,successful,16,US,1205.0
1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01,453.0,failed,40,US,453.0
1000034518,SPIN - Premium Retractable In-Ear Headphones with Mic,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24,8233.0,canceled,58,US,8233.0
100004195,STUDIO IN THE SKY - A Documentary Feature Film (Canceled),Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11,6240.57,canceled,43,US,6240.57


In [23]:
ks = ks16.union(ks18)

In [24]:
# Add the length of the project
ks = ks.withColumn('duration',datediff(ks.deadline,ks.launched))

In [25]:
display(ks.take(5))

ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,duration
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11,0.0,failed,0,GB,0.0,59
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12,220.0,failed,3,US,220.0,45
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17,1.0,failed,1,US,1.0,30
1000011046,Community Film Project: The Art of Neighborhood Filmmaking,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04,1283.0,canceled,14,US,1283.0,56
1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26,52375.0,successful,224,US,52375.0,35


In [26]:
ks.schema

Out[206]: StructType(List(StructField(ID,LongType,true),StructField(name,StringType,true),StructField(category,StringType,true),StructField(main_category,StringType,true),StructField(currency,StringType,true),StructField(deadline,DateType,true),StructField(goal,DoubleType,true),StructField(launched,DateType,true),StructField(pledged,DoubleType,true),StructField(state,StringType,true),StructField(backers,LongType,true),StructField(country,StringType,true),StructField(usd_pledged,DoubleType,true),StructField(duration,IntegerType,true)))

In [27]:
# Importing libraries to visualize the data
import matplotlib.pyplot as plt
import seaborn as sns
ksp=ks.toPandas()


/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [28]:
ksp.dtypes

Out[70]: ID int64
name object
category object
main_category object
currency object
deadline object
goal float64
launched object
pledged float64
state object
backers int64
country object
usd_pledged float64
duration int32
dtype: object

In [29]:
# Scientific Notations formatting
pd.options.display.float_format = '{:,.2f}'.format

In [30]:
# Show statistics of pandas dataframe
ksp.describe()

,ID,goal,pledged,backers,usd_pledged,duration
count,"694,188.00","694,188.00","694,188.00","694,188.00","694,188.00","694,188.00"
mean,"1,074,728,555.73","48,660.43","9,296.76",104.92,"7,409.81",34.63
std,"619,246,061.86","1,169,751.31","93,286.87",925.01,"81,477.01",68.75
min,"5,971.00",0.00,0.00,0.00,0.00,1.00
25%,"537,722,510.00","2,000.00",30.00,2.00,20.26,30.00
50%,"1,075,550,449.50","5,250.00",615.00,12.00,456.68,30.00
75%,"1,610,536,778.25","16,000.00","4,022.07",56.00,"3,262.08",38.00
max,"2,147,476,221.00","100,000,000.00","20,338,986.27","219,382.00","20,338,986.27","16,739.00"


In [31]:
# Distinct values for each column
for i in ks.columns:
    print(i, ks.select(i).distinct().count())

ID 374864
name 372071
category 159
main_category 15
currency 14
deadline 3164
goal 8414
launched 3169
pledged 62363
state 5
backers 3981
country 22
usd_pledged 142863
duration 99

In [32]:
# Explore the total amount of usd pledged for different countries
ks_usd=ks.select('country','usd_pledged').groupby('country').sum('usd_pledged').sort('country',ascending=False).toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [33]:
ks_usd.head()

,country,sum(usd_pledged)
0,US,"4,398,992,294.77"
1,GB,"356,535,857.28"
2,CA,"136,365,845.13"
3,AU,"58,896,499.85"
4,DE,"41,134,654.81"


In [34]:
ks_usd.columns=['country','total usd pledged']

In [35]:
fig,z=plt.subplots()
z=sns.barplot(x='country', y='total usd pledged', data=ks_usd).set_title("Country and Total usd pledged distribution in billions")
plt.ylim(0,4500000000)
plt.show()
# Y axis which is sum(usd pledged) is in bilions and we can see that US has the highest amount pledged around 4.4 billions followed by GB which is just 0.3 billion

In [36]:
ks_state=ks.groupby('state').count().sort('count',ascending=True).toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [37]:
ks_state

,state,count
0,suspended,3320
1,live,7225
2,canceled,71089
3,successful,246827
4,failed,365727


In [38]:
s=sns.barplot(x='state', y='count', data=ks_state).set_title("Project state distribution")
plt.show()

In [39]:
from pyspark.sql.functions import *
ks_backers=ks.select('main_category','backers').groupby('main_category').agg(count('backers')).sort('count(backers)',ascending=False).toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [40]:
ks_backers

,main_category,count(backers)
0,Film & Video,119473
1,Music,93864
2,Publishing,73105
3,Games,63225
4,Technology,58671
5,Design,53928
6,Art,52111
7,Food,45794
8,Fashion,41190
9,Theater,20873


In [41]:
fig,b=plt.subplots()
b=sns.barplot(x='count(backers)', y='main_category', data=ks_backers).set_title("Project category distribution by no. of backers ")
plt.show()

In [42]:
ksyear = ks.withColumn("launched", ks['launched'].cast('string'))
ksyear = ks.withColumn("launched", ks['launched'].substr(0,4))

In [43]:
pd.options.display.float_format = '{:,.2f}'.format

In [44]:
ks_year=ksyear.select('usd_pledged','launched').groupby('launched').sum('usd_pledged').sort('launched').toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [45]:
fig,c=plt.subplots()
c=sns.lineplot(x='launched', y='sum(usd_pledged)', data=ks_year).set_title("USD distribution as per the year")
display(fig)

#We 

In [46]:
#Now, we will analyse the distribution of successful and failed projects by main categories

failed = ksp.loc[ksp.state=='failed']
successful = ksp.loc[ksp.state=='successful']


In [47]:
fig,ax=plt.subplots(1,2)
sns.barplot(x=successful.main_category.value_counts().values,
            y=successful.main_category.value_counts().index, data=successful,ax=ax[0]).set_title("Dist. for successful projects")
sns.barplot(x=failed.main_category.value_counts().values,
            y=failed.main_category.value_counts().index, data=failed,ax=ax[1]).set_title("Dist. for failed projects")
plt.subplots_adjust(wspace=0.8)
plt.subplots_adjust(bottom=0.1, top=0.90)
display(fig)

In [48]:
#Distribution according to main category of Successful projects according to different columns

ksp['duration(days)'] = (ksp['deadline'] - ksp['launched']).dt.days


In [49]:
mean_ks=successful.groupby(['main_category']).mean()
mean_ks

,ID,goal,pledged,backers,usd_pledged,duration,duration(days)
main_category,,,,,,,
Art,"1,080,772,661.90","4,588.90","7,438.90",87.69,"5,746.16",30.56,30.56
Comics,"1,059,946,725.41","5,681.61","12,014.88",235.04,"10,075.17",32.24,32.24
Crafts,"1,079,588,146.82","3,290.66","6,401.40",94.55,"4,418.12",29.12,29.12
Dance,"1,079,644,681.23","4,727.84","5,331.03",63.53,"4,651.40",31.86,31.86
Design,"1,078,844,382.40","17,309.04","66,792.01",619.87,"48,076.10",33.51,33.51
Fashion,"1,067,020,313.68","10,347.23","22,848.05",221.93,"16,672.02",31.43,31.43
Film & Video,"1,072,617,776.05","11,297.72","14,134.45",152.91,"12,921.87",32.41,32.41
Food,"1,077,180,200.66","11,944.37","17,567.45",180.00,"15,054.37",31.59,31.59
Games,"1,077,355,922.51","15,694.10","55,528.41",833.12,"45,585.35",30.21,30.21


In [50]:
fig,ax=plt.subplots(1,2,figsize=(25,10))
sns.barplot(x=mean_ks.backers.values,
            y=mean_ks.backers.index, data=mean_ks, ax=ax[0]).set_title("Dist. as per avg backers")
sns.barplot(x=mean_ks['usd pledged'].values,
            y=mean_ks['usd pledged'].index, data=mean_ks, ax=ax[1]).set_title("Dist. as per avg pledged")
plt.subplots_adjust(wspace=0.8)
plt.subplots_adjust(bottom=0.2, top=0.90)
display(fig)

--------------------------------------------------------------------------- 
 KeyError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2656 try : 
 -> 2657 return self . _engine . get_loc ( key ) 
 2658 except KeyError : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'usd pledged'

During handling of the above exception, another exception occurred:

 KeyError Traceback (most recent call last)
 <command-4367245092359200> in <module> 
 2 sns.barplot(x=mean_ks.backers.values,
 3 y=mean_ks.backers.index, data=mean_ks, ax=ax[0]).set_title("Dist. as per avg backers")
 ----> 4 sns.barplot(x=mean_ks['usd pledged'].values,
 5 y=mean_ks['usd pledged'].index, data=mean_ks, ax=ax[1]).set_title("Dist. as per avg pledged")
 6 plt . subplots_adjust ( wspace = 0.8 ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/frame.py in __getitem__ (self, key) 
 2925 if self . columns . nlevels > 1 : 
 2926 return self . _getitem_multilevel ( key ) 
 -> 2927 indexer = self . columns . get_loc ( key ) 
 2928 if is_integer ( indexer ) : 
 2929 indexer = [ indexer ] 

 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2657 return self . _engine . get_loc ( key ) 
 2658 except KeyError : 
 -> 2659 return self . _engine . get_loc ( self . _maybe_cast_indexer ( key ) ) 
 2660 indexer = self . get_indexer ( [ key ] , method = method , tolerance = tolerance ) 
 2661 if indexer . ndim > 1 or indexer . size > 1 : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'usd pledged'

In [51]:
fig,d=plt.subplots()
sns.barplot(x=mean_ks.duration.values,
            y=mean_ks.duration.index, data=mean_ks).set_title("Dist. of avg. project duration")
display(fig)

In [52]:
mean_f=failed.groupby(['main_category']).mean()
mean_f

,ID,goal,pledged,backers,usd_pledged,duration,duration(days)
main_category,,,,,,,
Art,"1,073,367,658.36","66,405.07",644.58,8.94,588.05,34.10,34.10
Comics,"1,063,390,632.97","45,115.75","1,026.88",21.26,926.49,36.88,36.88
Crafts,"1,076,045,361.53","13,627.92",386.18,5.99,280.75,32.38,32.38
Dance,"1,067,779,509.75","16,120.86",644.29,9.65,584.98,34.73,34.73
Design,"1,078,147,053.87","61,111.69","3,172.78",32.27,"2,519.62",34.68,34.68
Fashion,"1,070,673,706.44","29,741.76",910.76,9.92,732.17,33.22,33.22
Film & Video,"1,073,626,532.84","118,038.52","1,572.22",15.29,"1,430.10",37.05,37.05
Food,"1,083,999,764.99","62,228.99","1,113.40",12.75,985.00,34.79,34.79
Games,"1,071,356,906.32","63,271.86","2,470.48",45.17,"2,125.37",34.08,34.08


In [53]:
# Import the required libraries

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [54]:
ks_dt=ks.select('main_category','goal','state','backers','country','usd_pledged','duration').filter("state== 'successful' or state== 'failed'")
ks_dt.show()

+-------------+--------+----------+-------+-------+-----------+--------+
main_category| goal| state|backers|country|usd_pledged|duration|
+-------------+--------+----------+-------+-------+-----------+--------+
 Publishing| 1000.0| failed| 0| GB| 0.0| 59|
 Film & Video| 45000.0| failed| 3| US| 220.0| 45|
 Music| 5000.0| failed| 1| US| 1.0| 30|
 Food| 50000.0|successful| 224| US| 52375.0| 35|
 Food| 1000.0|successful| 16| US| 1205.0| 20|
 Food| 25000.0| failed| 40| US| 453.0| 45|
 Publishing| 2500.0| failed| 0| CA| 0.0| 30|
 Music| 12500.0|successful| 100| US| 12700.0| 30|
 Crafts| 5000.0| failed| 0| US| 0.0| 30|
 Games|200000.0| failed| 0| US| 0.0| 45|
 Design| 2500.0| failed| 11| US| 664.0| 30|
 Comics| 1500.0| failed| 16| US| 395.0| 30|
 Publishing| 3000.0| failed| 20| US| 789.0| 30|
 Music| 250.0|successful| 7| US| 250.0| 15|
 Food| 5000.0| failed| 40| US| 1781.0| 30|
 Fashion| 20000.0|successful| 624| US| 34268.0| 35|
 Theater| 3500.0| failed| 12| US| 650.0| 30|
 Food| 500.0| failed| 3| NO| 6.18000816| 30|
 Comics| 175.0|successful| 66| US| 701.66| 20|
 Music| 10000.0|successful| 147| US| 15827.0| 30|
+-------------+--------+----------+-------+-------+-----------+--------+
only showing top 20 rows

In [55]:
# Create a 70-30 train test split

train_data,test_data=ks_dt.randomSplit([0.7,0.3])

In [56]:
train_data.show(5)
test_data.show(5)

+-------------+----+----------+-------+-------+------------+--------+
main_category|goal| state|backers|country| usd_pledged|duration|
+-------------+----+----------+-------+-------+------------+--------+
 Art| 1.0|successful| 17| GB|228.26419409| 30|
 Art| 1.0|successful| 37| US| 796.0| 30|
 Art| 1.0|successful| 78| US| 834.0| 18|
 Art| 3.0|successful| 7| US| 79.0| 30|
 Art| 4.0|successful| 12| US| 205.0| 30|
+-------------+----+----------+-------+-------+------------+--------+
only showing top 5 rows

+-------------+----+----------+-------+-------+-----------+--------+
main_category|goal| state|backers|country|usd_pledged|duration|
+-------------+----+----------+-------+-------+-----------+--------+
 Art| 1.0|successful| 1| US| 1.0| 33|
 Art| 5.0| failed| 2| US| 2.0| 30|
 Art| 5.0|successful| 50| US| 121.0| 3|
 Art|10.0|successful| 7| US| 17.0| 15|
 Art|20.0|successful| 3| US| 22.0| 20|
+-------------+----+----------+-------+-------+-----------+--------+
only showing top 5 rows

In [57]:
# Use StringIndexer to convert the categorical columns to hold numerical data

main_category_indexer = StringIndexer(inputCol='main_category',outputCol='main_category_index',handleInvalid='keep')
state_indexer = StringIndexer(inputCol='state',outputCol='state_index',handleInvalid='keep')
country_indexer = StringIndexer(inputCol='country',outputCol='country_index',handleInvalid='keep')

In [58]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=['main_category_index','country_index','goal','backers','usd_pledged','duration'],
                            outputCol='features')

In [59]:
# Create an object for the Decision Tree model
dt_model = DecisionTreeClassifier(labelCol='state_index',maxBins=5000)

In [60]:
# Pipeline is used to pass the data through indexer and assembler simultaneously in order to specify our machine learning algorithm workflow.

pipe = Pipeline(stages=[main_category_indexer,state_indexer,country_indexer,assembler,dt_model])

In [61]:
fit_model=pipe.fit(train_data)

In [62]:
results = fit_model.transform(test_data)

In [63]:
results.show(5)

+-------------+----+----------+-------+-------+-----------+--------+-------------------+-----------+-------------+--------------------+--------------------+--------------------+----------+
main_category|goal| state|backers|country|usd_pledged|duration|main_category_index|state_index|country_index| features| rawPrediction| probability|prediction|
+-------------+----+----------+-------+-------+-----------+--------+-------------------+-----------+-------------+--------------------+--------------------+--------------------+----------+
 Art| 1.0|successful| 1| US| 1.0| 33| 5.0| 1.0| 0.0|[5.0,0.0,1.0,1.0,...| [3257.0,543.0,0.0]|[0.85710526315789...| 0.0|
 Art| 5.0| failed| 2| US| 2.0| 30| 5.0| 0.0| 0.0|[5.0,0.0,5.0,2.0,...| [3257.0,543.0,0.0]|[0.85710526315789...| 0.0|
 Art| 5.0|successful| 50| US| 121.0| 3| 5.0| 1.0| 0.0|[5.0,0.0,5.0,50.0...|[1333.0,27520.0,0.0]|[0.04619970193740...| 1.0|
 Art|10.0|successful| 7| US| 17.0| 15| 5.0| 1.0| 0.0|[5.0,0.0,10.0,7.0...| [3257.0,543.0,0.0]|[0.85710526315789...| 0.0|
 Art|20.0|successful| 3| US| 22.0| 20| 5.0| 1.0| 0.0|[5.0,0.0,20.0,3.0...| [667.0,2280.0,0.0]|[0.22633186291143...| 1.0|
+-------------+----+----------+-------+-------+-----------+--------+-------------------+-----------+-------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [64]:
display(results.select(['state_index','prediction']).take(5))

state_index,prediction
1.0,0.0
0.0,0.0
1.0,1.0
1.0,0.0
1.0,1.0


In [65]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [66]:
ACC_evaluator = MulticlassClassificationEvaluator(
    labelCol="state_index", predictionCol="prediction", metricName="accuracy")

In [67]:
dtModel = fit_model.stages[4]
print(dtModel) 
accuracy = ACC_evaluator.evaluate(results)
print("Accuracy = %g" % (accuracy))
print("Test Error = %g" % (1.0 - accuracy))

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_cb782515d30a) of depth 5 with 45 nodes
Accuracy = 0.954508
Test Error = 0.0454924

In [68]:
fit_model.stages[-1].featureImportances

Out[161]: SparseVector(6, {1: 0.0034, 2: 0.1873, 3: 0.6005, 4: 0.2088})

In [69]:
display(fit_model.stages[-1])

treeNode
"{""index"":19,""featureType"":""continuous"",""prediction"":null,""threshold"":17.5,""categories"":null,""feature"":3,""overflow"":false}"
"{""index"":13,""featureType"":""continuous"",""prediction"":null,""threshold"":850.5,""categories"":null,""feature"":2,""overflow"":false}"
"{""index"":7,""featureType"":""continuous"",""prediction"":null,""threshold"":195.025,""categories"":null,""feature"":4,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":199.5,""categories"":null,""feature"":2,""overflow"":false}"
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":18.00302172,""categories"":null,""feature"":4,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":8.5,""categories"":null,""feature"":3,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [70]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [71]:
ExtractFeatureImp(fit_model.stages[-1].featureImportances, results , "features")

,idx,name,vals,score
1,3,backers,NaN,0.60
2,4,usd_pledged,NaN,0.21
0,2,goal,NaN,0.19
5,1,country_index,"[US, GB, CA, AU, DE, NL, FR, IT, ES, SE, NZ, D...",0.00
3,5,duration,NaN,0.00
4,0,main_category_index,"[Film & Video, Music, Publishing, Games, Techn...",0.00


In [72]:
display(ks.limit(5))

ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,duration
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11,0.0,failed,0,GB,0.0,59
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12,220.0,failed,3,US,220.0,45
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17,1.0,failed,1,US,1.0,30
1000011046,Community Film Project: The Art of Neighborhood Filmmaking,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04,1283.0,canceled,14,US,1283.0,56
1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26,52375.0,successful,224,US,52375.0,35


In [73]:

# Getting Numerical Columns from the Spark Dataframe
num_cols=['goal','backers','usd_pledged','duration']

In [74]:
categoricalcols=[]
for col in ks_dt.columns:
    if col in num_cols:
        pass
    else:
        categoricalcols.append(col)

In [75]:
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer

In [76]:
# Converting categorical columns to numeric using string indexer
flag=0
for col in categoricalcols:
    if flag != 1:
        indexer = StringIndexer(inputCol=col, outputCol=col+"_categorical").fit(ks_dt)
        indexed_df = indexer.transform(ks_dt)
        flag=1
    else:
        indexer = StringIndexer(inputCol=col, outputCol=col+"_categorical").fit(indexed_df)
        indexed_df = indexer.transform(indexed_df)

In [77]:
display(indexed_df.take(5))

main_category,goal,state,backers,country,usd_pledged,duration,main_category_categorical,state_categorical,country_categorical
Publishing,1000.0,failed,0,GB,0.0,59,2.0,0.0,1.0
Film & Video,45000.0,failed,3,US,220.0,45,0.0,0.0,0.0
Music,5000.0,failed,1,US,1.0,30,1.0,0.0,0.0
Food,50000.0,successful,224,US,52375.0,35,7.0,1.0,0.0
Food,1000.0,successful,16,US,1205.0,20,7.0,1.0,0.0


In [78]:
indexed_df.printSchema()

root
-- main_category: string (nullable = true)
-- goal: double (nullable = true)
-- state: string (nullable = true)
-- backers: long (nullable = true)
-- country: string (nullable = true)
-- usd_pledged: double (nullable = true)
-- duration: integer (nullable = true)
-- main_category_categorical: double (nullable = false)
-- state_categorical: double (nullable = false)
-- country_categorical: double (nullable = false)

In [79]:
indexed_df=indexed_df.drop('main_category','state','country')

In [80]:
pd.options.display.float_format = '{:,.2f}'.format

In [81]:
# Common statistics of all the columns
indexed_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
goal,612554,43156.51625628107,1085174.8589868369,0.0,1.0E8
backers,612554,114.44249486575877,980.4828413222216,0,219382
usd_pledged,612554,8098.233586188493,86037.88362806017,0.0,2.033898627E7
duration,612554,34.06963794212429,12.793770766391441,1,92
main_category_categorical,612554,4.184708613444692,3.654474808862858,0.0,14.0
state_categorical,612554,0.4029473319903225,0.4904907464806698,0.0,1.0
country_categorical,612554,0.6935943606604479,2.2036375703228135,0.0,21.0


In [82]:
#Creating a Correlation Dictionary to select the best positively and negatively Correlated values.
correlation_dict={}

for col in indexed_df.columns:
    correlation_dict[col]=indexed_df.corr(col1='backers',col2=col)

In [83]:
#https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
import operator
sorted_dict = sorted(correlation_dict.items(), key=operator.itemgetter(1),reverse=True)

In [84]:
sorted_dict[0:1][0][0]

Out[177]: 'backers'

In [85]:
sorted_dict[-6:]

Out[178]: [('usd_pledged', 0.7066878827476579),
 ('state_categorical', 0.12147979834269823),
 ('goal', 0.005371637370506434),
 ('duration', -0.0002708365498221666),
 ('main_category_categorical', -0.0027432480606117797),
 ('country_categorical', -0.009210399836692492)]

In [86]:
# Creating new_col_list which joins the slices created above.
new_col_list=sorted_dict[0:1]+sorted_dict[-6:]

In [87]:
collist=[]
for item in new_col_list:
    if item[0]=='state':
        continue
    else:
        collist.append(item[0])

In [88]:
# Getting Correlation within the columns of collist. 
f=indexed_df.select(collist).toPandas().corr()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [89]:
#Creating heatmap for visualizing the correlated columns so that we can eliminate auto-correlation.

import seaborn as sns
import matplotlib.pyplot as plt

fig,ax=plt.subplots()
plt.figure(figsize=(10,8))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
sns.heatmap(data=f)
display(fig)

In [90]:
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

columns = ['col1','col2','col3']

myGraph=spark.createDataFrame([(1.3,2.1,3.0),
                               (2.5,4.6,3.1),
                               (6.5,7.2,10.0)],
                              columns)
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=['col1','col2','col3'], 
                            outputCol=vector_col)
myGraph_vector = assembler.transform(myGraph).select(vector_col)
matrix = Correlation.corr(myGraph_vector, vector_col)

In [91]:
matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()
print(corrmatrix)

[[1.0, 0.9582184104641529, 0.9780872729407004], [0.9582184104641529, 1.0, 0.8776695567739841], [0.9780872729407004, 0.8776695567739841, 1.0]]

In [92]:
def plot_corr_matrix(correlations,attr,fig_no):
    fig=plt.figure(fig_no)
    ax=fig.add_subplot(111)
    ax.set_title("Correlation Matrix for Specified Attributes")
    ax.set_xticklabels(['']+attr)
    ax.set_yticklabels(['']+attr)
    cax=ax.matshow(correlations,vmax=1,vmin=-1)
    fig.colorbar(cax)
    plt.show()

In [93]:
plot_corr_matrix(corrmatrix, columns, 234)

/databricks/python/lib/python3.7/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance. In a future version, a new instance will always be created and returned. Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
 "Adding an axes using the same arguments as a previous axes "